In [4]:
text = """The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound."""

In [6]:
!pip install nltk

In [8]:
import nltk

nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
import re
import heapq
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

# Downloads (first time only)
nltk.download("punkt")
nltk.download("stopwords")

def summarize_nltk(text, ratio=0.3):
    # clean a bit
    clean = re.sub(r"\s+", " ", text).strip()

    sentences = sent_tokenize(clean)
    words = word_tokenize(clean.lower())

    stop_words = set(stopwords.words("english"))
    words = [w for w in words if w.isalpha() and w not in stop_words]

    # word frequencies
    freq = {}
    for w in words:
        freq[w] = freq.get(w, 0) + 1

    # normalize
    if not freq:
        return clean
    max_f = max(freq.values())
    for w in freq:
        freq[w] /= max_f

    # sentence scores
    sent_scores = {}
    for sent in sentences:
        ws = word_tokenize(sent.lower())
        score = 0.0
        for w in ws:
            if w in freq:
                score += freq[w]
        if score > 0:
            sent_scores[sent] = score

    k = max(1, int(len(sentences) * ratio))
    best = heapq.nlargest(k, sent_scores, key=sent_scores.get)

    # keep original order
    best_set = set(best)
    summary = " ".join([s for s in sentences if s in best_set])
    return summary

summary_nltk = summarize_nltk(text, ratio=0.30)
print("=== NLTK SUMMARY ===\n")
print(summary_nltk)

=== NLTK SUMMARY ===

The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Altho

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

# If needed in Colab:
# !pip -q install spacy
# !python -m spacy download en_core_web_sm

def load_spacy_model():
    try:
        return spacy.load("en_core_web_sm")
    except:
        raise RuntimeError("spaCy model 'en_core_web_sm' not found. Run: python -m spacy download en_core_web_sm")

nlp = load_spacy_model()

def summarize_spacy(text, ratio=0.3):
    doc = nlp(text)

    # word frequencies
    freq = {}
    for token in doc:
        if token.text.lower() not in STOP_WORDS and token.text.lower() not in punctuation and token.is_alpha:
            key = token.lemma_.lower()
            freq[key] = freq.get(key, 0) + 1

    if not freq:
        return text

    max_f = max(freq.values())
    for w in freq:
        freq[w] /= max_f

    # sentence scores
    sent_scores = {}
    sents = list(doc.sents)
    for sent in sents:
        score = 0.0
        for token in sent:
            key = token.lemma_.lower()
            if key in freq:
                score += freq[key]
        if score > 0:
            sent_scores[sent] = score

    k = max(1, int(len(sents) * ratio))
    best = heapq.nlargest(k, sent_scores, key=sent_scores.get)

    # preserve order
    best_set = set(best)
    summary = " ".join([sent.text.strip() for sent in sents if sent in best_set])
    return summary

summary_spacy = summarize_spacy(text, ratio=0.30)
print("=== SPACY SUMMARY ===\n")
print(summary_spacy)

=== SPACY SUMMARY ===

The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. According to Wayne Hale, a flight director

In [11]:
import re
import numpy as np


import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

def safe_sent_tokenize(t: str):
    try:
        return sent_tokenize(t)
    except LookupError:

        return t.split(". ")

def clean_ws(t: str) -> str:
    return re.sub(r"\s+", " ", t).strip()

def word_count(t: str) -> int:
    # рахунок "слів"
    tokens = re.findall(r"[A-Za-z]+", t)
    return len(tokens)

def sent_count(t: str) -> int:
    return len([s for s in safe_sent_tokenize(t) if s.strip()])

def stats(name: str, t: str):
    t = clean_ws(t)
    return {
        "name": name,
        "sentences": sent_count(t),
        "words": word_count(t),
        "chars": len(t),
    }

orig = stats("Original", text)
sp   = stats("SpaCy summary", summary_spacy)
nk   = stats("NLTK summary", summary_nltk)

def compression(a, b, key="words"):
    return b[key] / max(1, a[key])

rows = [orig, sp, nk]
for r in rows:
    print(r)

print("\nCompression vs Original:")
print(f"SpaCy  by words: {compression(orig, sp, 'words'):.2f} | by sentences: {compression(orig, sp, 'sentences'):.2f}")
print(f"NLTK   by words: {compression(orig, nk, 'words'):.2f} | by sentences: {compression(orig, nk, 'sentences'):.2f}")


{'name': 'Original', 'sentences': 16, 'words': 457, 'chars': 2906}
{'name': 'SpaCy summary', 'sentences': 5, 'words': 184, 'chars': 1168}
{'name': 'NLTK summary', 'sentences': 4, 'words': 193, 'chars': 1251}

Compression vs Original:
SpaCy  by words: 0.40 | by sentences: 0.31
NLTK   by words: 0.42 | by sentences: 0.25
